# 层和块
首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

#nn.Sequential定义了一种特殊的Module
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.2007,  0.1182, -0.0244, -0.2057,  0.0648,  0.2763, -0.0443, -0.0444,
         -0.0720, -0.0116],
        [-0.3654,  0.1575, -0.0320, -0.2817,  0.1067,  0.2986, -0.1514,  0.0854,
         -0.0809,  0.0304]], grad_fn=<AddmmBackward0>)

自定义块

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [6]:
net = MLP()
net(X)

tensor([[-0.0284,  0.1635,  0.0455,  0.2417, -0.2252,  0.0130,  0.2973, -0.1710,
          0.0131,  0.2003],
        [ 0.1655,  0.1086,  0.0373,  0.2900, -0.1829, -0.0269,  0.2272, -0.0335,
          0.0227,  0.1310]], grad_fn=<AddmmBackward0>)

顺序块,nn.Sequential的实现

In [7]:
class MySequential(nn.Module):
    #传入的是一个list
    def __init__(self,*args):
        super().__init__()
        for block in args:
            #_modules是一个特殊的成员变量，是一个ordered dict
            self._modules[block] = block
    
    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
            return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)


tensor([[ 0.0533,  0.3883, -0.2857, -0.7450, -0.0244,  0.1967,  0.4143,  0.2408,
         -0.2800,  0.3255, -0.3602,  0.3352,  0.7782, -0.2774,  0.0609,  0.3292,
         -0.0427,  0.0765, -0.1772,  0.0892,  0.4317, -0.3205, -0.1823,  0.2161,
         -0.3431, -0.3362, -0.1489, -0.2007,  0.3287,  0.4230,  0.4280, -0.9256,
         -0.8958,  0.1628,  0.1933,  0.3426,  0.3092, -0.1803,  0.0559,  0.0452,
         -0.5586,  0.3145,  0.1873,  0.6645,  0.2844, -0.6967,  0.1615, -0.6253,
          0.1016,  0.1273,  1.1094, -0.0088,  0.2430,  0.5048,  0.6715, -0.3076,
         -0.5530, -0.7185,  0.2848, -0.3977,  0.3643,  0.0758,  0.1437, -0.1576,
         -0.3006,  0.8629, -0.0731, -0.2644,  0.0671,  0.3337,  0.2578,  0.0794,
         -0.0065,  0.2746,  0.7139, -0.2100, -0.5257, -0.1971,  0.5081, -0.0463,
          0.2966, -0.0130, -0.0269,  0.1048, -0.1602,  0.3705, -0.5501, -0.0079,
         -0.2936, -0.1411, -0.1257, -0.1382, -0.0861,  0.4562,  0.3839, -0.1922,
         -0.1138, -0.1762,  

在正向传播函数中执行代码

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 随机的weight 不需要计算梯度
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.0925, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.2044, grad_fn=<SumBackward0>)